In [17]:
import math
import numpy as np
from scipy import special
import scipy.linalg as la

In [18]:
class basis_set():
    def __init__(self, coord, alpha, coeff):
        self.coord = np.array(coord)
        self.alpha = alpha
        self.coeff = coeff
        self.norm = ((2*alpha)/math.pi)**0.75

In [19]:
def gprod(c1,alpha1,c2,alpha2):
    p= alpha1 + alpha2
    q= (alpha1 * alpha2)/p
    Q= c1 - c2
    Q2 = np.dot(Q,Q)
    KAB= math.exp(-q*(Q2))
    return KAB

In [20]:
def overlap(molecule):
    
    Natoms = len(molecule)
    S = np.zeros([Natoms, Natoms])
    
    for i in range(Natoms):
        for j in range(Natoms):
            
            Nbasis_i = len(molecule[i])
            Nbasis_j = len(molecule[j])
            
            for k in range(Nbasis_i):
                for l in range(Nbasis_j):
                    E = gprod(molecule[i][k].coord, molecule[i][k].alpha, molecule[j][l].coord, molecule[j][l].alpha)
                    Norm = molecule[i][k].norm * molecule[j][l].norm
                    Pval = molecule[i][k].alpha + molecule[j][l].alpha
                    S[i,j] += E * Norm * molecule[i][k].coeff * molecule[j][l].coeff * (math.pi/Pval)**(3/2)
   
    return S           

In [21]:
def kinetic(molecule):
    
    Natoms=len(molecule)
    T = np.zeros([Natoms, Natoms])
    for i in range(Natoms):
        for j in range(Natoms):
            
            Nbasis_i = len(molecule[i])
            Nbasis_j = len(molecule[j])
            
            for k in range(Nbasis_i):
                for l in range(Nbasis_j):
                    
                    Const = molecule[i][k].coeff * molecule[j][l].coeff
                    E = gprod(molecule[i][k].coord, molecule[i][k].alpha, molecule[j][l].coord, molecule[j][l].alpha)
                    Norm = molecule[i][k].norm * molecule[j][l].norm
                    Pval = molecule[i][k].alpha + molecule[j][l].alpha
                    s = E * Norm * Const * (math.pi/Pval)**(3/2)
                    
                    Px = (molecule[i][k].alpha*np.array(molecule[i][k].coord)+molecule[j][l].alpha*np.array(molecule[j][l].coord))
                    Pv = Px/Pval
                    Pg = Pv - np.array(molecule[j][l].coord)
                    Pgx2 = Pg[0]*Pg[0]
                    Pgy2 = Pg[1]*Pg[1]
                    Pgz2 = Pg[2]*Pg[2]
                    
                    T[i,j] += 3 * molecule[j][l].alpha * s
                    T[i,j] -= 2 * molecule[j][l].alpha * molecule[j][l].alpha * (Pgx2 + (0.5/Pval)) * s
                    T[i,j] -= 2 * molecule[j][l].alpha * molecule[j][l].alpha * (Pgy2 + (0.5/Pval)) * s
                    T[i,j] -= 2 * molecule[j][l].alpha * molecule[j][l].alpha * (Pgz2 + (0.5/Pval)) * s
                        
    return T

In [22]:
def boys(n, x):
    if x==0:
        B = 1.0/(2*n + 1)
    else:
        B = special.gamma(n+0.5) * special.gammainc(n+0.5, x) / (2*x**(n+0.5))
        
    return B 

In [23]:
def nuclear_attraction(molecule, atom_coordinates, Z):
    
    Natoms = len(molecule)
    Nnucleus = len(Z)
    V= np.zeros([Natoms,Natoms])
    for atoms in range(Nnucleus):
        for i in range(Natoms):
            for j in range(Natoms):
                
                Nprimitives_i = len(molecule[i])
                Nprimitives_j = len(molecule[j])
                
                for k in range(Nprimitives_i):
                    for l in range(Nprimitives_j):
                        
                        p = molecule[i][k].alpha + molecule[j][l].alpha
                        P = ((molecule[i][k].alpha * molecule[i][k].coord) + (molecule[j][l].alpha * molecule[j][l].coord))/p
                        RPA = P - atom_coordinates[atoms] 
                        RPA2 = np.dot(RPA, RPA)
                        E = gprod(molecule[i][k].coord, molecule[i][k].alpha, molecule[j][l].coord, molecule[j][l].alpha)
                        const =  molecule[i][k].coeff * molecule[j][l].coeff
                        N = molecule[i][k].norm * molecule[j][l].norm
                        
                        V[i, j] += -Z[atoms]*E*((2*math.pi)/p) * boys(0, p*RPA2) * const * N
   
    return V

In [24]:
def electron_repulsion(molecule):
    Natoms = len(molecule)
    gabcd = np.zeros([Natoms,Natoms,Natoms,Natoms])
    for i in range(Natoms):
        Nprimitives_i = len(molecule[i])
        for ni in range(Nprimitives_i):            
            for j in range(Natoms):
                Nprimitives_j = len(molecule[j])
                for nj in range(Nprimitives_j):
                    for k in range(Natoms):   
                        for nk in range(Nprimitives_i):
                            for l in range(Natoms):
                                for nl in range(Nprimitives_j):
                                    aa = molecule[i][ni].alpha                                    
                                    ab = molecule[j][nj].alpha
                                    ac = molecule[k][nk].alpha
                                    ad = molecule[l][nl].alpha
                                    
                                    p = aa + ab
                                    P = ((molecule[i][ni].coord * aa) + (molecule[j][nj].coord* ab))/p
                                    Eab = gprod(molecule[i][ni].coord, aa, molecule[j][nj].coord, ab)
                                    Nab = molecule[i][ni].norm * molecule[j][nj].norm
                                    Cab = molecule[i][ni].coeff * molecule[j][nj].coeff
                                    A_ab = Eab * Nab * Cab
                                    
                                    pp = ac + ad
                                    PP = ((molecule[k][nk].coord * ac)+(molecule[l][nl].coord * ad))/pp
                                    Ecd = gprod(molecule[k][nk].coord, ac, molecule[l][nl].coord, ad)
                                    Ncd = molecule[k][nk].norm * molecule[l][nl].norm
                                    Ccd = molecule[k][nk].coeff * molecule[l][nl].coeff         
                                    A_cd = Ecd * Ncd * Ccd
                                    
                                    RPPP = P - PP
                                    RPPP2 = np.dot(RPPP, RPPP)
                                    alpha = pp*p/(pp+p)
                                    
                                    gabcd[i,j,k,l] += A_ab * A_cd * boys(0, alpha*RPPP2)*(2*(math.pi)**(2.5))/(p * pp * (p+pp)**(0.5))
                                    
    return gabcd

In [25]:
def nuclear_repulsion(Z, atom_coordinates):
    natoms = len(Z)
    nuc_energy = 0
    if (natoms == 1):
        nuc_energy = 0
    else:
        for i in range(1, natoms):
            distance = atom_coordinates[i] - atom_coordinates[i-1]
            distance2 = np.dot(distance, distance)
            distancesq = np.sqrt(distance2)
            nuc_energy += Z[i]*Z[i-1]/distancesq
            
    return nuc_energy            

In [26]:
def Fini(F):
    Fo = np.transpose(Sh) @ F @ Sh
    return Fo

#This is to tranform the resulting eigenvectors into the original(non-orthonormal) basis    
def Cmat(M):
    Foeig = la.eigh(M)
    C = np.dot (Sh, Foeig[1])
    return C,Foeig[0]

#This is to construct density matrix and store the older values for convergence test
def Dmat(C,P):
    m = len(C)
    OldD = np.zeros((m,m))
    for i in range(m):    
        for j in range(m): 
            OldD[i,j]=P[i,j]
            P[i,j] = 0.0
            for s in range(0,Ne//2):
                P[i,j] = P[i,j] + C[i,s]*C[j,s]
    return P,OldD

def Hartree(H, P, Gabcd):
    m = len(H)
    J = np.zeros((m,m))  
    for i in range(0,m):  
        for j in range(0,m):  
            J[i,j] = H[i,j]  
            for k in range(0,m):  
                for l in range(0,m):  
                    J[i,j] = J[i,j] + P[k,l]*2*Gabcd[i,j,k,l]
    return J

#It calculates delta value used for the test of convergence
def Delta(P,OldD):
    delta = 0.0e0  
    for i in range(0,m):  
        for j in range(0,m):  
            delta = delta+((P[i,j]-OldD[i,j])**2)  
    delta = (delta/4)**(0.5)
    return delta

#It calculates the electronic energy
def Eelec(P, H, F):
    E = 0.0  
    for i in range(0,m):  
        for j in range(0,m):  
            E = E + P[i,j]*(H[i,j] + F[i,j])  
    return E  

In [27]:
# Check the phia = lambda x,y,z : phia(x, y, z) + ax(x) * ay(y) *az(z) line, it shows RecursionError: maximum recursion depth exceeded
# Can output be in the form of an array storing functions i.e output.insert(i, phia) becomes output[i]= phia. It is showing errors with this.
def Basis_Func(molecule):
    natoms = len(molecule)
    output = []
    for i in range(natoms):
        nbasis = len(molecule[i])
        phia = lambda x,y,z: 0
        for j in range(nbasis):
            ax = lambda x: math.exp(-molecule[i][j].alpha * (x - molecule[i][j].coord[0])**2)
            ay = lambda y: math.exp(-molecule[i][j].alpha * (y - molecule[i][j].coord[1])**2)
            az = lambda z: math.exp(-molecule[i][j].alpha * (z - molecule[i][j].coord[2])**2)
            phia = lambda x,y,z : phia(x, y, z) + ax(x) * ay(y) *az(z) * molecule[i][j].norm *molecule[i][j].coeff
        output.insert(i, phia)
    return output 

In [28]:
def Rho(P, atom_coordinates, Ne, phia):
    dx = 0.05
    nb = len(atom_coordinates)
    X = []
    for i in range(nb):
        X.append(atom_coordinates[i][0])
    max_X = max(X)
    min_X = min(X)
    
    Y = []
    for i in range(nb):
        Y.append(atom_coordinates[i][1])
    max_Y = max(Y)
    min_Y = min(Y)
    
    Z = []
    for i in range(nb):
        Z.append(atom_coordinates[i][2])
    max_Z = max(Z)
    min_Z = min(Z)
        
    x = np.arange((min_X - 2),(max_X +2),(dx))
    y = np.arange((min_Y - 2),(max_Y +2),(dx))
    z = np.arange((min_Z - 2),(max_Z +2),(dx))
    
    [Xgrid, Ygrid, Zgrid] = np.meshgrid(x,y,z)
    
    Rho = np.zeros(len(Xgrid))
    for i in range(nb):
        for j in range(nb):
            Rho = Rho + 2*P[i,j]*phia[i](Xgrid, Ygrid, Zgrid)* phia[j](Xgrid, Ygrid, Zgrid)

            
    return Rho, Xgrid, Ygrid, Zgrid

In [29]:
# Does np.sum is able to sum overall the X elements followed by sum over Y elements and Z. Check E_x[i,j] line
def Exchange(Rho, Xgrid, Ygrid, Zgrid, phia):
    dx = 0.05
    nb = len(phia)
    rs = (3 /(4* math.pi* Rho))**(1/3)
    E_x = np.zeros([nb, nb])
    for i in range(nb):
        for j in range(nb):
            
            E_x[i,j]= np.sum(np.sum(np.sum(phia[i](Xgrid, Ygrid, Zgrid)* (0.6109/rs)* phia[j](Xgrid, Ygrid, Zgrid))))*dx**3
    
    return E_x

In [30]:
def Correlation(Rho, Xgrid, Ygrid, Zgrid, phia):
    dx = 0.05
    nb = len(phia)
    rs = (3 /(4* math.pi* Rho))**(1/3)
    x = rs**(0.5)
    A = 0.0621814
    xO = -0.409286
    b = 13.0720
    c = 42.7198
    
    X = x**2 + b*x + c
    X_xO = xO**2 + b*xO + c
    Q = np.sqrt(4*c - b**2)
    E_C = ((A/2)*((math.log((x**2)/X)) + 2*b/Q* math.atan(Q/(2*x+b)) - 
            b*(xO/X_xO)*(math.log((x-xO)**2/X)+ 2*((b+2*xO)/Q)* math.atan(Q/(2*x+b)))))
    
    V_C = E_C - (1/6)*A*(c*(x-xO) - b*x*xO)/((x-xO)*(x**2 + b*x + c))
    V_C_Mat = np.zeros([nb, nb])
    for i in range(nb):
        for j in range(nb):
            
            V_C_Mat[i,j] = np.sum(np.sum(np.sum(phia[i](Xgrid, Ygrid, Zgrid)* V_C * phia[j](Xgrid, Ygrid, Zgrid))))*dx**3
            
    return V_C_Mat   

In [31]:
def Evaluate_Exc(Rho,dx):
    rs = (3 /(4* math.pi* Rho))**(1/3)
    dx = 0.05
    E_x = np.sum(np.sum(np.sum(-0.4582*Rho/rs)))*dx**3
    x = rs**(0.5)
    A = 0.0621814
    xO = -0.409286
    b = 13.0720
    c = 42.7198
    X = x**2 + b*x + c
    X_xO = xO**2 + b*xO + c
    Q = np.sqrt(4*c - b**2)
    e_C = ((A/2)*((math.log((x**2)/X)) + 2*b/Q* math.atan(Q/(2*x+b)) - 
            b*(xO/X_xO)*(math.log((x-xO)**2/X)+ 2*((b+2*xO)/Q)* math.atan(Q/(2*x+b)))))
    
    E_c = np.sum(np.sum(np.sum(Rho* e_C)))* dx**3
    
    E_xc = E_x + E_c
    
    return E_xc

In [32]:
H1_1a = basis_set([0,0,0], 0.3425250914E+01, 0.1543289673E+00)
H1_1b = basis_set([0,0,0], 0.6239137298E+00, 0.5353281423E+00)
H1_1c = basis_set([0,0,0], 0.1688554040E+00, 0.4446345422E+00)
He2_1a = basis_set([1.7565,0,0], 0.6362421394E+01, 0.1543289673E+00)
He2_1b = basis_set([1.7565,0,0], 0.1158922999E+01, 0.5353281423E+00)
He2_1c = basis_set([1.7565,0,0], 0.3136497915E+00, 0.4446345422E+00)

H1_1s = [H1_1a, H1_1b, H1_1c]
He2_1s = [He2_1a, He2_1b, He2_1c]

Z = [1.0, 2.0]
atom_coordinates = [np.array([0,0,0]), np.array([1.7565,0,0])]
molecule = [H1_1s, He2_1s]

V = nuclear_attraction(molecule, atom_coordinates, Z)
T = kinetic(molecule)

m = len(molecule)
Ne = 2

H=np.zeros((m,m))

for i in range(m):    
    for j in range(m): 
        H[i,j] = V[i,j] + T[i,j]

        
phia = Basis_Func(molecule)
S = overlap(molecule)
eigval = la.eigh(S)
Pi = eigval[1]
Di = np.diag(eigval[0].real**(-0.5))
Sh = np.dot(Pi, np.dot(Di, np.transpose(Pi))) 

Gabcd = electron_repulsion(molecule)
Enuc = nuclear_repulsion(Z, atom_coordinates)


delta =1.0
convergence=0.00000001 

i=0
#to define maximum number of iterations
maxit=20
P = np.zeros((m,m))

while(i<maxit):
    i +=1
    
    if (delta>convergence):
        
        J = Hartree(H, P, Gabcd)
        [Rho, Xgrid, Ygrid, Zgrid] = Rho(P, atom_coordinates, Ne, phia)
        V_X = Exchange(Rho, Xgrid, Ygrid, Zgrid, phia)
        V_C = Correlation(Rho, Xgrid, Ygrid, Zgrid, phia)
        F = H + J + V_X + V_C
        Fi=Fini(F)
        #print('Fi',i,'=',Fi)
        C,Foeig=Cmat(Fi)
        #print('C',i,'=',C)
        #print('C',i,'=',Foeig)
        P,OldD=Dmat(C,P)
        #print('Do',i,'=',P)
        #print('Dold',i,'=',OldD)
        delta=Delta(P,OldD)
        #print('delta=',delta)
        E=Eelec(P, H, H+J) + Evaluate_Exc(Rho,dx)
        print('E',i,'=',E+Enuc)
        continue
    
    else:
        print('Convergence achieved!')
        break

RecursionError: maximum recursion depth exceeded